# AI Applications - Mini Project 2
> By Oliver Dietsche & Simon Peier

## Imports

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image

## Dataset
We decided to use part of the [Tiny ImageNet](https://www.kaggle.com/c/tiny-imagenet) dataset. 

According to the description in Kaggle, the Tiny ImageNet dataset has the following properties: "Tiny ImageNet contains 200 classes for training. Each class has 500 images. The test set contains 10,000 images. All images are 64x64 colored ones." It fulfills all the required key characteristics, except that it has too many classes (and thus too many samples). To meet all criteria, we pre-processed the data-set and chose 6 classes to use. As the dataset contains lots of classes of different categories, we settled for classes from the animal realm.

### Dataset import

In [ ]:
data_dir = "dataset"
words_file = os.path.join(data_dir, "words.txt")
image_size = (64, 64)

# Load words.txt
with open(words_file, 'r') as f:
    class_labels = {}
    for line in f:
        line = line.strip().split('\t')
        class_labels[line[0]] = line[1]

label_to_int_map = {label: i for i, label in enumerate(class_labels.values())}

def label_to_int(label):
    return label_to_int_map[label]

def int_to_label(intVal):
    return list(label_to_int_map.keys())[intVal]

# Load images and labels
images = []
labels = []

for folder_name in os.listdir(data_dir):
    if folder_name.startswith('n'):
        label = class_labels[folder_name]
        image_folder_path = os.path.join(data_dir, folder_name, "images")
        for image_name in os.listdir(image_folder_path):
            image_path = os.path.join(image_folder_path, image_name)
            image = img_to_array(load_img(image_path))
            images.append(image)
            labels.append(label)

images = np.array(images)
labels = np.array(labels)

### Normalization

In [ ]:
print(f"Range of pixel values: [{images.min()};{images.max()}]")

images = images / 255.0

print(f"Range of pixel values: [{images.min()};{images.max()}]")

### Dataset split

In [ ]:
# split into test and training data
train_val_imgs, test_imgs, train_val_labels, test_labels = train_test_split(images, labels, test_size=0.3)

In [ ]:
# split into training and validation data
train_imgs, val_imgs, train_labels, val_labels = train_test_split(train_val_imgs, train_val_labels, test_size=0.2)

### Visualization

In [ ]:
# Collect and print image stats
(total_samples, image_height, image_width, num_channels) = images.shape
test_samples = test_imgs.shape[0]
train_samples = train_imgs.shape[0]
val_samples = val_imgs.shape[0]
print(f"Total samples:       {total_samples}")
print(f"Testing samples:     {test_samples}")
print(f"Training samples:    {train_samples}")
print(f"Validation samples:  {val_samples}\n")
print(f"Image height:        {image_height}")
print(f"Image width:         {image_width}")
print(f"Number of channels:  {num_channels}")

# Count samples per class
samples_per_class = {}
for label in labels:
    samples_per_class[label] = samples_per_class.get(label, 0) + 1

# Determine if dataset is balanced
class_labels = list(samples_per_class.keys())
class_counts = list(samples_per_class.values())
plt.bar(class_labels, class_counts)
plt.title('Samples per Class')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.show()

# Plot sample image for every class
num_samples_to_plot = 6
classes_to_plot = np.random.choice(list(samples_per_class.keys()), num_samples_to_plot, replace=False)

fig, axs = plt.subplots(1, num_samples_to_plot, figsize=(15, 3))

for i, class_label in enumerate(classes_to_plot):
    class_indices = np.where(labels == class_label)[0]
    sample_index = np.random.choice(class_indices)
    sample_image = images[sample_index]
    axs[i].imshow(sample_image)
    axs[i].set_title(class_label)
    axs[i].axis('off')

plt.show()

## Underfitting
### Model definition

In [ ]:
num_classes = len(class_labels)

model = models.Sequential([
    layers.Conv2D(8, 3, padding='same', activation='relu', strides=3, input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D(3),
    layers.Flatten(),
    layers.Dense(num_classes)
])

model.summary()

### Training

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

train_labels_int = np.array([label_to_int(l) for l in train_labels])
val_labels_int = np.array([label_to_int(l) for l in val_labels])

history = model.fit(train_imgs, train_labels_int, epochs=100, 
                    validation_data=(val_imgs, val_labels_int))

### Plotting

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 2])
plt.legend(loc='lower right')
plt.show()

### Interpretation

- Both the training and validation accuracy have a big increase in accuracy in the first few epochs but it quickly stagnates around 60% accuracy. This is an indicator that the model might not be able to capture the complexity our categorization within the defined layers.
- The loss graph also performs very poorly, with training and validation separating over time instead of growing towards eachother.

In [ ]:
predictions = model.predict(val_imgs)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = [int_to_label(l) for l in predicted_labels]

labels = np.unique(val_labels)

fig, axes = plt.subplots(1, 4, figsize=(18, 6), constrained_layout=True)

norm_options = [None, 'true', 'pred', 'all']

for ax, norm in zip(axes, norm_options):
    cm = confusion_matrix(val_labels, predicted_labels, normalize=norm)
    
    disp = ax.matshow(cm, cmap='Blues')
    ax.set_title(f'Normalization: {str(norm)}')
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(np.arange(len(labels)))
    ax.set_yticklabels(labels)

    for i in range(len(labels)):
        for j in range(len(labels)):
            text = ax.text(j, i, f'{cm[i, j]:.2f}',
                           ha="center", va="center", color="black")

fig.colorbar(disp, ax=axes, orientation='vertical')
plt.show()

TODO: add interpretation of matrices

## Overfitting
### Model definition

In [ ]:
overfitting_model = models.Sequential([
    layers.Conv2D(48, 4, activation='relu', padding='same', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D(2),
    layers.Conv2D(64, 4, activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.Conv2D(96, 4, activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 4, activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(48),
    layers.Dense(num_classes, activation='softmax')
])

overfitting_model.summary()

### Training

In [ ]:
overfitting_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = overfitting_model.fit(train_imgs, train_labels_int, epochs=30, 
                    validation_data=(val_imgs, val_labels_int))

### Plotting

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 5])
plt.legend(loc='lower right')
plt.show()

In [ ]:
predictions = overfitting_model.predict(val_imgs)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = [int_to_label(l) for l in predicted_labels]

labels = np.unique(val_labels)

fig, axes = plt.subplots(1, 4, figsize=(18, 6), constrained_layout=True)

norm_options = [None, 'true', 'pred', 'all']

for ax, norm in zip(axes, norm_options):
    cm = confusion_matrix(val_labels, predicted_labels, normalize=norm)
    
    disp = ax.matshow(cm, cmap='Blues')
    ax.set_title(f'Normalization: {str(norm)}')
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(np.arange(len(labels)))
    ax.set_yticklabels(labels)

    for i in range(len(labels)):
        for j in range(len(labels)):
            text = ax.text(j, i, f'{cm[i, j]:.2f}',
                           ha="center", va="center", color="black")

fig.colorbar(disp, ax=axes, orientation='vertical')
plt.show()

### Discussion
Our model behaves as predicted. 

The training accuracy increases continuously, reaching very high values, as the model memorizes the training data and is able to classify it accurately. The validation accuracy on the other hand increases initially, but after a certain time it plateaus, because the model becomes too specialized to the training data.

The training loss steadily decreases up until a certain point when it reaches very low values. This indicates, that the model is fitting the data extremely well. However the validation loss, after initially decreasing, begins to increase once the model starts overfitting.


## Optimizing
### Model definition

In [ ]:
optimized_model = models.Sequential([
    layers.Conv2D(48, 4, kernel_regularizer=l2(0.001), activation='relu', padding='same', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D(2),
    layers.Dropout(0.25),
    layers.Conv2D(64, 4, kernel_regularizer=l2(0.001), activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.Dropout(0.25),
    layers.Conv2D(96, 4, kernel_regularizer=l2(0.001), activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.Dropout(0.25),
    layers.Conv2D(128, 4, kernel_regularizer=l2(0.001), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(48, kernel_regularizer=l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(num_classes, kernel_regularizer=l2(0.001), activation='softmax'),
])

optimized_model.summary()

In [ ]:
optimized_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = optimized_model.fit(train_imgs, train_labels_int, epochs=40, 
                    validation_data=(val_imgs, val_labels_int))

### Plotting

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 5])
plt.legend(loc='lower right')
plt.show()

In [ ]:
predictions = optimized_model.predict(val_imgs)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = [int_to_label(l) for l in predicted_labels]

labels = np.unique(val_labels)

fig, axes = plt.subplots(1, 4, figsize=(18, 6), constrained_layout=True)

norm_options = [None, 'true', 'pred', 'all']

for ax, norm in zip(axes, norm_options):
    cm = confusion_matrix(val_labels, predicted_labels, normalize=norm)
    
    disp = ax.matshow(cm, cmap='Blues')
    ax.set_title(f'Normalization: {str(norm)}')
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=90)
    ax.set_yticks(np.arange(len(labels)))
    ax.set_yticklabels(labels)

    for i in range(len(labels)):
        for j in range(len(labels)):
            text = ax.text(j, i, f'{cm[i, j]:.2f}',
                           ha="center", va="center", color="black")

fig.colorbar(disp, ax=axes, orientation='vertical')
plt.show()